In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pheochromocytoma_and_Paraganglioma/GSE19422'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of pheochromocytoma (PCC)/paraganglioma (PGL) tumors"
!Series_summary	"Transcriptional analysis of 84 primary pheochromocytoma (PCC)/paraganglioma tumors."
!Series_overall_design	"84 samples (primary pheochromocytoma (PCC)/paraganglioma tumors) were hybridized onto a cDNA microarray in order to investigate possible heterogeneity within these tumors"
Sample Characteristics Dictionary:
{0: ['tissue type: PCC primary tumor', 'tissue type: PGL primary tumor', 'tissue type: Normal adrenal tissue']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True

# Define data availability keys for traits, age and gender
trait_row = 0
age_row = None
gender_row = None

# Define data type conversion functions
def convert_trait(value):
    value = value.split(":")[1].strip().lower()
    if value == "pcc primary tumor":
        return 1
    elif value == "pgl primary tumor":
        return 1
    elif value == "normal adrenal tissue":
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    value = value.split(":")[1].strip().lower()
    if value in ["male", "m"]:
        return 1
    elif value in ["female", "f"]:
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE19422', './preprocessed/Pheochromocytoma_and_Paraganglioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if necessary
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Pheochromocytoma_and_Paraganglioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/trait_data/GSE19422.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM482992': [1], 'GSM482993': [1], 'GSM482994': [1], 'GSM482995': [1], 'GSM482996': [1], 'GSM482997': [1], 'GSM482998': [1], 'GSM482999': [1], 'GSM483000': [1], 'GSM483001': [1], 'GSM483002': [1], 'GSM483003': [1], 'GSM483004': [1], 'GSM483005': [1], 'GSM483006': [1], 'GSM483007': [1], 'GSM483008': [1], 'GSM483009': [1], 'GSM483010': [1], 'GSM483011': [1], 'GSM483012': [1], 'GSM483013': [1], 'GSM483014': [1], 'GSM483015': [1], 'GSM483016': [1], 'GSM483017': [1], 'GSM483018': [1], 'GSM483019': [1], 'GSM483020': [1], 'GSM483021': [1], 'GSM483022': [1], 'GSM483023': [1], 'GSM483024': [1], 'GSM483025': [1], 'GSM483026': [1], 'GSM483027': [1], 'GSM483028': [1], 'GSM483029': [1], 'GSM483030': [1], 'GSM483031': [1], 'GSM483032': [1], 'GSM483033': [1], 'GSM483034': [1], 'GSM483035': [1], 'GSM483036': [1], 'GSM483037': [1], 'GSM483038': [1], 'GSM483039': [1], 'GSM483040': [1], 'GSM483041': [1], 'GSM483042': [1], 'GSM483043': [1], 'GSM483044': [1], 'GSM483045': [1], 'GSM483046': [1], 'GSM48304

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/gene_data/GSE19422.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Pheochromocytoma_and_Paraganglioma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE19422', './preprocessed/Pheochromocytoma_and_Paraganglioma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/GSE19422.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Pheochromocytoma_and_Paraganglioma', the least common label is '0.0' with 6 occurrences. This represents 6.67% of the dataset.
The distribution of the feature 'Pheochromocytoma_and_Paraganglioma' in this dataset is fine.

